<h1>Federated Learning - GTEx_V8 Example</h1>


<h2>Import dependencies</h2>

In [1]:
#dependencies for helper functions/classes
import pandas as pd
import pyarrow.parquet as pq
from typing import NamedTuple
import os.path as path
import os
import progressbar
import requests
import numpy as np
import random


#keras for ML
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dropout, Input, Dense
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.utils import plot_model, normalize
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import SGD, Adam, Nadam, Adadelta
from tensorflow.keras.activations import relu, elu, sigmoid

#sklearn for preprocessing the data and train-test split
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score, classification_report
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, r2_score, mean_squared_error, mean_absolute_error

#for plots
import matplotlib
import matplotlib.pyplot as plt

#%matplotlib inline

### # Parameter cell -->

In [7]:
seed = 7
_test_size = 0.1

In [8]:
class Labels(NamedTuple):
    '''
    One-hot labeled data
    '''
    tissue: np.ndarray
    sex: np.ndarray
    age: np.ndarray
    death: np.ndarray
        

class Genes:
    '''
    Class to load GTEX samples and gene expressions data
    '''
    def __init__(self, samples_path: str = '', expressions_path: str = '', problem_type: str = "classification"):
        self.__set_samples(samples_path)
        self.__set_labels(problem_type)
        if expressions_path != '':
            self.expressions = self.get_expressions(expressions_path)

    def __set_samples(self, sample_path: str) -> pd.DataFrame:
        self.samples: pd.DataFrame = pq.read_table(sample_path).to_pandas()
        self.samples["Death"].fillna(-1.0, inplace = True)
        self.samples: pd.DataFrame = self.samples.set_index("Name")
        self.samples["Sex"].replace([1, 2], ['male', 'female'], inplace=True)
        self.samples["Death"].replace([-1,0,1,2,3,4], ['alive/NA', 'ventilator case', '<10 min.', '<1 hr', '1-24 hr.', '>1 day'], inplace=True)
        self.samples = self.samples[~self.samples['Death'].isin(['>1 day'])]
        return self.samples

    def __set_labels(self, problem_type: str = "classification") -> Labels:
        self.labels_list = ["Tissue", "Sex", "Age", "Death"]
        self.labels: pd.DataFrame = self.samples[self.labels_list]
        self.drop_list = self.labels_list + ["Subtissue", "Avg_age"]
        
        if problem_type == "classification":
            dummies_df = pd.get_dummies(self.labels["Age"])
            print(dummies_df.columns.tolist())
            self.Y = dummies_df.values
        
        if problem_type == "regression":
            self.Y = self.samples["Avg_age"].values
        
        return self.Y

    def sex_output(self, model):
        return Dense(units=self.Y.sex.shape[1], activation='softmax', name='sex_output')(model)

    def tissue_output(self, model):
        return Dense(units=self.Y.tissue.shape[1], activation='softmax', name='tissue_output')(model)

    def death_output(self, model):
        return Dense(units=self.Y.death.shape[1], activation='softmax', name='death_output')(model)

    def age_output(self, model):
        '''
        Created an output layer for the keras mode
        :param model: keras model
        :return: keras Dense layer
        '''
        return Dense(units=self.Y.age.shape[1], activation='softmax', name='age_output')(model)


    def get_expressions(self, expressions_path: str)->pd.DataFrame:
        '''
        load gene expressions DataFrame
        :param expressions_path: path to file with expressions
        :return: pandas dataframe with expression
        
        '''
        selected_genes = list(pd.read_csv('../data/gtex/gain_selected_features_tissues_False.csv')['ids'].values)
        
        if expressions_path.endswith(".parquet"):
            return pq.read_table(expressions_path).to_pandas().set_index("Name") #[selected_genes]
        else:
            separator = "," if expressions_path.endswith(".csv") else "\t"
            return pd.read_csv(expressions_path, sep=separator).set_index("Name") #[selected_genes]

    def prepare_data(self, normalize_expressions: bool = True)-> np.ndarray:
        '''
        :param normalize_expressions: if keras should normalize gene expressions
        :return: X array to be used as input data by keras
        '''
        data = self.samples.join(self.expressions, on = "Name", how="inner")
        ji = data.columns.drop(self.drop_list)
        x = data[ji]
        
        # adding one-hot-encoded tissues and sex
        #x = pd.concat([x,pd.get_dummies(data['Tissue'], prefix='tissue'), pd.get_dummies(data['Sex'], prefix='sex')],axis=1)
        
        steps = [('standardization', StandardScaler()), ('normalization', MinMaxScaler())]
        pre_processing_pipeline = Pipeline(steps)
        transformed_data = pre_processing_pipeline.fit_transform(x)

        x = transformed_data
        
        print('Data length', len(x))
        
        return x #normalize(x, axis=0) if normalize_expressions else x
    
    def get_features_dataframe(self, add_tissues=False):
        data = self.samples.join(self.expressions, on = "Name", how="inner")
        ji = data.columns.drop(self.drop_list)
        df = data[ji]
        if add_tissues:
            df = pd.concat([df,pd.get_dummies(data['Tissue'], prefix='tissue'), pd.get_dummies(data['Sex'], prefix='sex')],axis=1)
        x = df.values
        
        min_max_scaler = MinMaxScaler()
        x_scaled = min_max_scaler.fit_transform(x)
        df_normalized = pd.DataFrame(x_scaled, columns=df.columns, index=df.index)
        return df_normalized


In [9]:
samples_path = '../data/gtex/v8_samples.parquet'
expressions_path = '../data/gtex/v8_expressions.parquet'

In [6]:
genes = Genes(samples_path, expressions_path, problem_type="regression")
X = genes.prepare_data(True)
Y = genes.Y

Data length 15343


In [10]:
#split data into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y, 
                                                    test_size=_test_size, 
                                                    random_state=seed)

In [11]:
def create_clients(image_list, label_list, num_clients=10, initial='clients'):
    ''' return: a dictionary with keys clients' names and value as 
                data shards - tuple of images and label lists.
        args: 
            image_list: a list of numpy arrays of training images
            label_list:a list of binarized labels for each image
            num_client: number of fedrated members (clients)
            initials: the clients'name prefix, e.g, clients_1 
            
    '''

    #create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]

    #randomize the data
    data = list(zip(image_list, label_list))
    random.shuffle(data)

    #shard data and place at each client
    size = len(data)//num_clients
    shards = [data[i:i + size] for i in range(0, size*num_clients, size)]

    #number of clients must equal number of shards
    assert(len(shards) == len(client_names))

    return {client_names[i] : shards[i] for i in range(len(client_names))} 

In [12]:
#create clients
clients = create_clients(X_train, y_train, num_clients=3, initial='client')

In [13]:
clients.keys(), clients['client_1'][0][0].shape

(dict_keys(['client_1', 'client_2', 'client_3']), (18388,))

In [14]:
def batch_data(data_shard, bs=32):
    '''Takes in a clients data shard and create a tfds object off it
    args:
        shard: a data, label constituting a client's data shard
        bs:batch size
    return:
        tfds object'''
    #seperate shard into data and labels lists
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(bs)

In [15]:
#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)
    
#process and batch the test set  
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

In [17]:
from keras import backend as K

def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

def optimized_age_model_regression():
    # optimized_age_model(x_train, x_val, y_train, y_val, params: dict):
    input_layer = Input(shape=(clients['client_1'][0][0].shape[0],))
    reg = keras.regularizers.l1_l2(l1=0.3, l2=0.3)
    mod = Dense(1024, activation=relu)(input_layer) # 196
    mod = Dropout(0.1)(mod) 
    mod = Dense(512, activation=relu)(mod) # 196
    mod = Dropout(0.1)(mod)    
    mod = Dense(64, activation=relu)(mod) #64
    mod = Dropout(0.1)(mod)
    
    outputs = [Dense(1, name='age_output')(mod)] #let's try to make it simple and start with age 
    #outputs = [Dense(y_train.shape[1], activation='sigmoid', name='age_output')(mod)] #let's try to make it simple and start with age 
    loss = {'age_output': 'mse'}
    weights={'age_output': 1.0}
    metrics = {'age_output': ['mae', coeff_determination]}
    
    model = Model(inputs=input_layer, outputs=outputs)
    model.summary()
    model.compile(optimizer='adam',
              loss=loss,
              loss_weights=weights,
              metrics=metrics,
                 )

    return model


In [18]:
def Huber(yHat, y, delta=1.):
    return np.where(np.abs(y-yHat) < delta,.5*(y-yHat)**2 , delta*(np.abs(y-yHat)-0.5*delta))

def transform_to_probas(age_intervals):
    class_names = ['20-29', '30-39', '40-49', '50-59', '60-69', '70-79']
    res = []
    for a in age_intervals:
        non_zero_index = class_names.index(a)
        res.append([0 if i != non_zero_index else 1 for i in range(len(class_names))])
    return np.array(res)
    
def transform_to_interval(age_probas):
    class_names = ['20-29', '30-39', '40-49', '50-59', '60-69', '70-79']
    return np.array(list(map(lambda p: class_names[np.argmax(p)], age_probas)))        

In [19]:
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad


# def test_model(X_test, Y_test,  model, comm_round):
#     cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
#     #logits = model.predict(X_test, batch_size=100)
#     logits = model.predict(X_test)
#     loss = cce(Y_test, logits)
#     acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
#     print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
#     return acc, loss

In [68]:
rmse = []
mae = []
r2 = []
huber_loss = []
comms_round = 2
    
global_model = optimized_age_model_regression()
    
#commence global training loop
for comm_round in range(comms_round):
            
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)
    
    #loop through each client and create new local model
    for client in client_names:
#         smlp_local = SimpleMLP()
#         local_model = smlp_local.build(784, 10)
#         local_model.compile(loss=loss, 
#                       optimizer=optimizer, 
#                       metrics=metrics)

        
        local_model = optimized_age_model_regression()
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        
        local_model.fit(clients_batched[client], epochs=1, verbose=1)
        predictions = local_model.predict(X_test)
        test_y = y_test

        print("R^2", r2_score(test_y, predictions))
        print("Mean squared error", mean_squared_error(test_y, predictions))
        print("Mean absolute error", mean_absolute_error(test_y, predictions))
        print('Huber loss', np.mean(Huber(test_y, predictions)))

        rmse.append(mean_squared_error(test_y, predictions))
        mae.append(mean_absolute_error(test_y, predictions))
        r2.append(r2_score(test_y, predictions))
        huber_loss.append(np.mean(Huber(test_y, predictions)))
        #fit local model with client's data
#         local_model.fit(clients_batched[client], epochs=1, verbose=1)
        
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    #update global model 
#     global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
#     for(X_test, Y_test) in test_batched:
#         global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)


Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 18388)]           0         
_________________________________________________________________
dense_12 (Dense)             (None, 1024)              18830336  
_________________________________________________________________
dropout_12 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_13 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 64)                32832     
_________________________________________________________________
dropout_14 (Dropout)         (None, 64)                0   

KeyboardInterrupt: 

In [7]:
Y[0:100], min(Y), max(Y)

(array([54.5, 54.5, 54.5, 54.5, 54.5, 54.5, 54.5, 54.5, 54.5, 54.5, 54.5,
        54.5, 54.5, 54.5, 54.5, 54.5, 54.5, 54.5, 54.5, 64.5, 64.5, 64.5,
        64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5,
        64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5,
        64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5,
        64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5,
        64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5,
        64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5,
        64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5,
        64.5]),
 24.5,
 74.5)

In [8]:
X.shape, Y.shape, Y[0]

((15343, 18388), (15343,), 54.5)

In [9]:
from sklearn import model_selection
X_1, X_2, y_1, y_2 = model_selection.train_test_split(X, Y, test_size=0.5, random_state=seed)

In [10]:
y_1[8].item()

64.5

In [11]:
_dtype = np.float32
y_1 = np.vstack(y_1).astype(np.float32)
y_2 = np.vstack(y_2).astype(np.float32)
X_1.dtype, y_1.dtype,X_2.dtype, y_2.dtype

(dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'))

In [12]:
sy.version.__version__

'0.2.8'

<h2>Setup config</h2>
Init hook, connect with grid nodes, etc...

In [14]:
hook = sy.TorchHook(torch)

compute_nodes = []
for node in nodes:
    # For syft 0.2.8 --> replace DynamicFLClient with DataCentricFLClient
    compute_nodes.append( DataCentricFLClient(hook, node) )

In [15]:
compute_nodes

[<Federated Worker id:h1>, <Federated Worker id:h2>]

## 1 - Conversion to Tensor

The code below will convert GTEx data samples to tensors.

In [13]:
shared_x1, shared_x2, shared_y1, shared_y2 = X_1, X_2, y_1, y_2

# Convert numpy array to torch tensors -->
shared_x1 = torch.from_numpy(shared_x1)
shared_x2 = torch.from_numpy(shared_x2)
shared_y1 = torch.from_numpy(shared_y1)
shared_y2 = torch.from_numpy(shared_y2)

shared_x1 = torch.tensor(shared_x1, dtype=torch.float32)
shared_x2 = torch.tensor(shared_x2, dtype=torch.float32)
shared_y1 = torch.tensor(shared_y1, dtype=torch.float32)
shared_y2 = torch.tensor(shared_y2, dtype=torch.float32)

datasets  = [shared_x1, shared_x2]
labels = [shared_y1, shared_y2]

/Users/pankajverma/anaconda3/envs/pysyft_v028/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/Users/pankajverma/anaconda3/envs/pysyft_v028/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/Users/pankajverma/anaconda3/envs/pysyft_v028/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by Inte

In [14]:
shared_y2[0], Y[0]

(tensor([64.5000]), 54.5)

# Below using centralized way (full data) --->

In [30]:
# Concatenate 
X = torch.cat((shared_x1, shared_x2), dim=0)
Y = torch.cat((shared_y1, shared_y2), dim=0)
rmse = []
mae = []
r2 = []
huber_loss = []

from torch import nn, optim
import torch.nn.functional as F
import torch

loss_func = torch.nn.MSELoss()

# TODO: Define your network architecture here
class Regression(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.fc1 = nn.Linear(num_features, 1024)
        self.dropout = nn.Dropout(0.1) 
        self.fc2 = nn.Linear(1024, 512)
        self.dropout = nn.Dropout(0.1) 
        self.fc3 = nn.Linear(512, 64)
        self.dropout = nn.Dropout(0.1) 
        self.fc4 = nn.Linear(64, 1)
        
    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        
        return x
    
# Create the network, define the criterion and optimizer
NUM_FEATURES = 18388
model = Regression(NUM_FEATURES)
# criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
    
epochs = 2

# for e in range(epochs):

#     epoch_loss = 0.0
#     epoch_acc = 0.0
#     optimizer.zero_grad()
    
#     pred = model(X)
# #     loss = F.cross_entropy(pred, Y)
#     loss = loss_func(pred, Y)

#     loss.backward()
#     optimizer.step()
    
#     y_pred = pred.detach().numpy()
#     y_true = Y.detach().numpy()
    
#     print("R^2", r2_score(y_true, y_pred))
#     print("Mean squared error", mean_squared_error(y_true, y_pred))
#     print("Mean absolute error", mean_absolute_error(y_true, y_pred))

#     rmse.append(mean_squared_error(y_true, y_pred))
#     mae.append(mean_absolute_error(y_true, y_pred))
#     r2.append(r2_score(y_true, y_pred))
      
#     print(f"Epoch: {e}",f"Training loss: {loss}")
    
# print('Mean MSE = ', np.mean(rmse))
# print('Mean MAE = ', np.mean(mae))
# print('Mean R2 = ', np.mean(r2))

In [31]:
data[0].shape, data[1].shape, data[0]

(torch.Size([7671, 18388]),
 torch.Size([7672, 18388]),
 tensor([[0.0149, 0.0696, 0.2304,  ..., 0.1199, 0.0991, 0.1440],
         [0.0000, 0.0322, 0.1970,  ..., 0.0884, 0.0457, 0.2258],
         [0.0142, 0.0029, 0.1234,  ..., 0.3109, 0.1795, 0.4097],
         ...,
         [0.0000, 0.0065, 0.1092,  ..., 0.2724, 0.1307, 0.3145],
         [0.0000, 0.0286, 0.1617,  ..., 0.1892, 0.0736, 0.4122],
         [0.0136, 0.0028, 0.2949,  ..., 0.2558, 0.2131, 0.2198]]))

In [32]:
def epoch_total_size(data):
    total = 0
    for i in range(len(data)):
        for j in range(len(data[i])):
            total += data[i][j].shape[0]
            
    return total

data = datasets
target = labels
opt = optimizer
N_EPOCS = 25
rmse = []
mae = []
r2 = []
huber_loss = []

def train(epoch):
    model.train()
    epoch_total = epoch_total_size(data)
    current_epoch_size = 0
    for i in range(len(data)):
        correct = 0

        epoch_loss = 0.0
        epoch_acc = 0.0
        current_epoch_size += len(data[i])

        opt.zero_grad()
        pred = model(data[i])
        loss = loss_func(pred, target[i])
        loss.backward()
        opt.step()

        y_pred = pred.detach().numpy()
        y_true = target[i].detach().numpy()

        rmse.append(mean_squared_error(y_true, y_pred))
        mae.append(mean_absolute_error(y_true, y_pred))
        r2.append(r2_score(y_true, y_pred))

        print('Train Epoch: {} | With h{} data |: Train Loss: {:.3f} | R^2: {:.3f} | Mean squared error: {:.3f} | Mean absolute error: {:.3f}'.format(
                  epoch, i+1, loss, r2_score(y_true, y_pred), mean_squared_error(y_true, y_pred), mean_absolute_error(y_true, y_pred)))

for epoch in range(N_EPOCS):
    train(epoch)

Train Epoch: 0 | With h1 data |: Train Loss: 2902.472 | R^2: -16.916 | Mean squared error: 2902.473 | Mean absolute error: 52.350
Train Epoch: 0 | With h2 data |: Train Loss: 2891.946 | R^2: -16.531 | Mean squared error: 2891.946 | Mean absolute error: 52.220
Train Epoch: 1 | With h1 data |: Train Loss: 2883.145 | R^2: -16.797 | Mean squared error: 2883.145 | Mean absolute error: 52.164
Train Epoch: 1 | With h2 data |: Train Loss: 2868.124 | R^2: -16.387 | Mean squared error: 2868.124 | Mean absolute error: 51.990
Train Epoch: 2 | With h1 data |: Train Loss: 2856.880 | R^2: -16.635 | Mean squared error: 2856.880 | Mean absolute error: 51.910
Train Epoch: 2 | With h2 data |: Train Loss: 2838.488 | R^2: -16.207 | Mean squared error: 2838.488 | Mean absolute error: 51.703
Train Epoch: 3 | With h1 data |: Train Loss: 2823.579 | R^2: -16.429 | Mean squared error: 2823.579 | Mean absolute error: 51.585
Train Epoch: 3 | With h2 data |: Train Loss: 2801.196 | R^2: -15.981 | Mean squared error:

<h2>2 - Tagging tensors</h2>
The code below will add a tag (of your choice) to the data that will be sent to grid nodes. This tag is important as the network will need it to retrieve this data later.

In [18]:
tag_input = []
tag_label = []

for i in range(len(compute_nodes)):
    tag_input.append(datasets[i].tag("#X", "#gtex_v8", "#dataset","#balanced").describe("The input datapoints to the GTEx_V8 dataset."))
    tag_label.append(labels[i].tag("#Y", "#gtex_v8", "#dataset","#balanced").describe("The input labels to the GTEx_V8 dataset."))

<h2> 3 - Sending our tensors to grid nodes</h2>

In [19]:
shared_x1 = tag_input[0].send(compute_nodes[0]) # First chunk of dataset to h1
shared_x2 = tag_input[1].send(compute_nodes[1]) # Second chunk of dataset to h2

shared_y1 = tag_label[0].send(compute_nodes[0]) # First chunk of labels to h1
shared_y2 = tag_label[1].send(compute_nodes[1]) # Second chunk of labels to h2

WebSocketConnectionClosedException: Connection is already closed.

In [27]:
print("X tensor pointers: ", shared_x1, shared_x2)
print("Y tensor pointers: ", shared_y1, shared_y2)

X tensor pointers:  (Wrapper)>[PointerTensor | me:70361333315 -> h1:35881159115]
	Tags: #balanced #dataset #gtex_v8 #X 
	Shape: torch.Size([900, 18420])
	Description: The input datapoints to the GTEx_V8 dataset.... (Wrapper)>[PointerTensor | me:49684275468 -> h2:52723085225]
	Tags: #balanced #dataset #gtex_v8 #X 
	Shape: torch.Size([900, 18420])
	Description: The input datapoints to the GTEx_V8 dataset....
Y tensor pointers:  (Wrapper)>[PointerTensor | me:49137384384 -> h1:4180671457]
	Tags: #balanced #Y #gtex_v8 #dataset 
	Shape: torch.Size([900])
	Description: The input labels to the GTEx_V8 dataset.... (Wrapper)>[PointerTensor | me:10561419581 -> h2:1629165329]
	Tags: #balanced #Y #gtex_v8 #dataset 
	Shape: torch.Size([900])
	Description: The input labels to the GTEx_V8 dataset....


<h2>Disconnect nodes</h2>

In [28]:
for i in range(len(compute_nodes)):
    compute_nodes[i].close()

### Go to the following address to search available tags:
http://0.0.0.0:5000/search-available-tags